In [1]:
from typing import Iterable
from sampo.schemas import Time
from sampo.scheduler.multi_agency.block_generator import generate_blocks, SyntheticBlockGraphType
from sampo.scheduler.multi_agency.multi_agency import StochasticManager, ScheduledBlock
from sampo.scheduler.multi_agency import Agent, validate_block_schedule
from uuid import uuid4
from copy import deepcopy

from sampo.generator import SimpleSynthetic
from sampo.scheduler import HEFTScheduler
from random import Random

r_seed = 231
rand = Random(r_seed)
ss = SimpleSynthetic(rand)

scheduler_constructors = [HEFTScheduler, HEFTScheduler, HEFTScheduler, HEFTScheduler, HEFTScheduler, HEFTScheduler, HEFTScheduler, HEFTScheduler]
base_contractor = ss.contractor(10)

bg = generate_blocks(SyntheticBlockGraphType.SEQUENTIAL, 100, [1, 1, 1], lambda x: (50, 60), 0.5, rand)

def finish_time(schedule: Iterable[ScheduledBlock]) -> Time:
    return max([sblock.end_time for sblock in schedule])

Can not find native module; switching to default
SEQUENTIAL SEQUENTIAL SEQUENTIAL PARALLEL GENERAL SEQUENTIAL SEQUENTIAL GENERAL SEQUENTIAL SEQUENTIAL PARALLEL


In [29]:
from sampo.schemas import WorkerProductivityMode, Contractor, IntervalGaussian
from sampo.schemas.time_estimator import DefaultWorkEstimator, WorkTimeEstimator
from sampo.scheduler.multi_agency import Manager
from typing import Callable


def construct_work_estimator(i: int, productivity: WorkerProductivityMode) -> WorkTimeEstimator:
    work_estimator = DefaultWorkEstimator()
    work_estimator.set_productivity_mode(productivity)
    for worker in ['driver', 'fitter', 'manager', 'handyman', 'electrician', 'engineer']:
        work_estimator.set_worker_productivity(IntervalGaussian(0.2 * i + 0.2, 1, 0, 2), worker)
    return work_estimator


def construct_agent(i: int, scheduler_constructor, contractor: Contractor, productivity: WorkerProductivityMode) -> Agent:
    work_estimator = construct_work_estimator(i, productivity)
    
    return Agent(f'Agent {i}', scheduler_constructor(work_estimator=work_estimator), [contractor])

def test_with_manager(manager_constructor: Callable[[list[Agent]], Manager], productivity: WorkerProductivityMode):
    contractors = [deepcopy(base_contractor) for _ in scheduler_constructors]
    for contractor in contractors:
        contractor.id = str(uuid4())
    
    agents = [construct_agent(i, scheduler_constructors[i % len(scheduler_constructors)], contractor, productivity)
              for i, contractor in enumerate(contractors)]
    manager = manager_constructor(agents)
    
    scheduled_blocks = manager.manage_blocks(bg)
    # validate_block_schedule(bg, scheduled_blocks, agents)
    
    return finish_time(scheduled_blocks.values())

In [39]:
time_static = test_with_manager(Manager, WorkerProductivityMode.Stochastic)
time_static

9633

In [40]:
time_stochastic = test_with_manager(StochasticManager, WorkerProductivityMode.Stochastic)
time_stochastic

4531

In [41]:
def run_comparison(iterations: int):
    successes = 0
    for i in range(iterations):
        time_static = test_with_manager(Manager, WorkerProductivityMode.Stochastic)
        time_stochastic = test_with_manager(StochasticManager, WorkerProductivityMode.Stochastic)
        print(f'{time_static} {time_stochastic}')
        if time_stochastic < time_static:
            successes += 1
    print(f'Success percentage: {int(100 * successes / iterations)}% on {iterations} iterations')

run_comparison(10)

27254 11829
16765 51175
19170 43856
28078 18893
15790 13870
29057 6302
12200 8015
18533 15938
5346 8329
11771 12090
Success percentage: 60% on 10 iterations


# Mode 2

In [15]:
test_with_manager(Manager, WorkerProductivityMode.Stochastic)

2540

In [51]:
def test_in_mode(mode: WorkerProductivityMode):
    conjuncted = bg.to_work_graph()
    print(f'Size: {conjuncted}')
    
    work_estimator = DefaultWorkEstimator()
    work_estimator.set_productivity_mode(mode)
    
    contractors = [deepcopy(base_contractor) for _ in scheduler_constructors]
    for contractor in contractors:
        contractor.id = str(uuid4())
    
    for worker in ['driver', 'fitter', 'manager', 'handyman', 'electrician', 'engineer']:
        for i, contractor in enumerate(contractors):
            work_estimator.set_worker_productivity(IntervalGaussian(0.2 * i + 0.2, 1, 0, 2), worker, contractor.id)
            
    agents = [Agent(f'Agent {i}', scheduler_constructors[i % len(scheduler_constructors)](work_estimator=work_estimator), [contractor])
              for i, contractor in enumerate(contractors)]
    manager = StochasticManager(agents)
    
    scheduled_blocks = manager.manage_blocks(bg)
    # validate_block_schedule(bg, scheduled_blocks, agents)
    
    ma_time = finish_time(scheduled_blocks.values())
    
    scheduler = HEFTScheduler(work_estimator=work_estimator)
    conjuncted_time = scheduler.schedule(conjuncted, [contractors[-1]]).execution_time
    return ma_time, conjuncted_time

In [49]:
# test_in_mode(WorkerProductivityMode.Static)

(1407, 1408)

In [53]:
test_in_mode(WorkerProductivityMode.Stochastic)

Size: WorkGraph(start=start, finish=finish, nodes=[start, 95d3f5fb-7472-413c-a12f-0086919c83fc, 16ab957b-8c28-485b-aed0-1857886c4881, 2a985e07-cfc2-4830-af2d-3fba7af47781, d3378ba0-aeaa-2548-fc77-9c910d5752bf, 2df0bd58-cb15-50f1-1a51-a86987e3d9cb, 65544c07-3ff7-f4c8-0d65-ee235c26cc48, 19799a61-846d-4aec-47cb-83df18bee394, d04ad2f2-5854-8205-a44b-3111aef44681, c141e388-813b-3bf8-5192-435fd3db2c15, b5a82e4d-a095-546d-5c76-62a4e419d99f, 8e23db52-a25d-4ff1-83ac-864a02a13ce9, f5d4a2fe-bbc8-4d5c-3edb-28b3ec9e8571, d0f282ce-5240-6555-8212-18c55ffe7ee2, a2c9d590-56d9-3111-1883-5d2946088d4a, 7e446c9f-1f91-5020-4e40-d81ff0cf0b15, 6a78b626-074b-9d7a-ee52-e7b701882dc0, 01c94a57-e9f2-7e68-5b19-99fbcb328c4d, 548fb1b5-68cd-6e99-60cb-711c0d9c2884, 0ffda816-7423-0cae-2500-864119392ced, b9041db1-3a26-808f-bc3f-54d8f5833bb1, ab1941bb-be3b-044d-59b6-2c02c1e96f08, 31b61e5b-5152-0c36-6f01-112568f166de, bbd9d1d8-4de3-ee61-fe83-aea90caba73b, 996ed174-fe4e-bb7f-fb36-a7389634fa56, 4fe68253-ec5c-0523-8ef4-71842c

TypeError: 'NoneType' object is not iterable

In [ ]:
for _ in range(10):
    print(test_in_mode(WorkerProductivityMode.Stochastic))